# Import Dependencies

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

# Import and preprocess Data

In [10]:
from kaggle.api import KaggleApi

In [17]:
api = KaggleApi()
# connect kaggle api with given username and key    
api.authenticate()

In [3]:
df = pd.read_csv(os.path.join('Data/data', 'train.csv', 'train.csv'))

In [108]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,good
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [5]:
from tensorflow.keras.layers import TextVectorization

X = df['comment_text']

# add a new column called good if the comment is not toxic
df['good'] = df['toxic'].apply(lambda x: 1 if x == 0 else 0)

In [6]:
# create y that will take the rest columns starting from 2
y = df.iloc[:, 2:].values

In [10]:
y.shape

(159571, 7)

In [8]:
X.shape

(159571,)

In [33]:
# create vectorizer
vectorizer = TextVectorization(max_tokens=20000,
                               output_mode='int',
                               output_sequence_length=67)
                               

In [34]:
vectorizer.adapt(X.values)

In [35]:
vectorizer_text  = vectorizer(X.values)

In [36]:
len(vectorizer.get_vocabulary())

20000

In [37]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=20000, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=67, # how long is each input
                             name="embedding_1") 

embedding

In [38]:
sample = embedding(vectorizer("hello world"))

In [56]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(7, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [40]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [24]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 20)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 20, 128)           2560000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 7)                 903       
                                                                 
Total params: 2560903 (9.77 MB)
Trainable params: 256

In [25]:
train_proportion = 0.7
val_proportion = 0.2
test_proportion = 0.1

# Calculate the number of samples for each set
num_samples = len(X)
num_train = int(num_samples * train_proportion)
num_val = int(num_samples * val_proportion)
num_test = num_samples - num_train - num_val

# Split the data and labels
X_train = X[:num_train]
y_train = y[:num_train]

X_val = X[num_train:num_train + num_val]
y_val = y[num_train:num_train + num_val]

X_test = X[num_train + num_val:]
y_test = y[num_train + num_val:]

In [41]:
model_1_history = model_1.fit(X_train, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              y_train,
                              epochs=3,
                              validation_data=(X_val, y_val))

Epoch 1/3
3491/3491 [==============================] - 140s 40ms/step - loss: 0.1311 - accuracy: 0.9104 - val_loss: 0.0935 - val_accuracy: 0.9309
Epoch 2/3
3491/3491 [==============================] - 144s 41ms/step - loss: 0.0829 - accuracy: 0.9417 - val_loss: 0.0818 - val_accuracy: 0.9454
Epoch 3/3
3491/3491 [==============================] - 140s 40ms/step - loss: 0.0715 - accuracy: 0.9526 - val_loss: 0.0791 - val_accuracy: 0.9494


In [82]:
input_padded = [["I fucking hate you i hope you die you fucking nigger"]]

In [83]:
predictions = model_1.predict(input_padded)

1/1 [==============================] - 0s 45ms/step


In [84]:
predictions

array([[0.54641366, 0.50254774, 0.46358237, 0.47973493, 0.48376662,
        0.5093317 , 0.50748366]], dtype=float32)

In [85]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(7, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)

In [89]:
model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])

In [87]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 67)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 67, 128)           2560000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 7)                 455   

In [48]:
vectorizer_text

<tf.Tensor: shape=(159571, 20), dtype=int64, numpy=
array([[ 645,   76,    2, ..., 6599,   16,   62],
       [   1,   54, 2489, ...,    0,    0,    0],
       [ 425,  441,   70, ...,  473,  101,    5],
       ...,
       [   1, 7392,  383, ...,    0,    0,    0],
       [   5,   12,  534, ...,  378,  137,   90],
       [   5,    8,  130, ...,   40,  420,    4]], dtype=int64)>

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    epochs=3, validation_data=(X_val,y_val))

In [94]:
model.save('Model/RNN_Model_Bidirectional')

INFO:tensorflow:Assets written to: Model/RNN_Model_Bidirectional\assets


INFO:tensorflow:Assets written to: Model/RNN_Model_Bidirectional\assets


In [95]:
model.evaluate(X_test,y_test,verbose=1)

499/499 [==============================] - 10s 20ms/step - loss: 0.0607 - accuracy: 0.9572


[0.06074872612953186, 0.9572001695632935]

In [98]:
test_str = ["Michael is such a nice person and a happy soul i really like him and he is a good friend of mine"]

In [119]:
probs = model.predict(test_str)

1/1 [==============================] - 0s 55ms/step


In [120]:
probs

array([[1.19827185e-02, 3.89421002e-05, 3.65210162e-03, 1.17149575e-04,
        1.03357444e-02, 4.29090636e-04, 9.88603473e-01]], dtype=float32)

In [121]:
model_1_preds = tf.squeeze(tf.round(probs))


In [122]:
model_1_preds

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 1.], dtype=float32)>

In [129]:
def print_labels(text):
    string = [text]
    prediction = model.predict(string)
    prediction = tf.round(prediction[0])
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "hate", "good"]
    for i, score in enumerate(prediction):
        if score == 1:
            print(labels[i])




print_labels("This is a good day ")


1/1 [==============================] - 0s 55ms/step
good
